Create a TS+biology tracers initial file for one bathymetry based on a restart file from a different bathymetry

In [1]:
import netCDF4 as nc
import numpy as np 

from salishsea_tools import nc_tools

%matplotlib inline

/home/sallen/anaconda/envs/py3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/sallen/anaconda/envs/py3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# New Bathymetry (via its mesh mask)

In [2]:
mesh = nc.Dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')
mbathy = mesh.variables['mbathy'][0,:,:] 
#used to calculate number of vertical ocean grid cells at each (i,j) (1=land point)
gdepw = mesh.variables['gdepw_0'][0,:,:,:]
surface_tmask = mesh.variables['tmask'][0,0,:,:]
surface_tmask = np.abs(surface_tmask-1)
tmask = mesh.variables['tmask'][0,:,:,:]
tmask = np.abs(tmask-1)
lats = mesh.variables['nav_lat'][:]
lons = mesh.variables['nav_lon'][:]
mesh.close()

In [3]:
# calculate bathymetry based on meshmask
NEMO_bathy = np.zeros(mbathy.shape)
for i in range(NEMO_bathy.shape[1]):
    for j in range(NEMO_bathy.shape[0]):
        level = mbathy[j,i]
        NEMO_bathy[j,i] = gdepw[level,j,i]
NEMO_bathy = np.ma.masked_array(NEMO_bathy, mask = surface_tmask)

# Old Bathymetry (based on its mesh mask)

In [4]:
oldmesh = nc.Dataset('/home/sallen/MEOPAR/NEMO-forcing/grid/mesh_mask201702.nc')
oldmbathy =oldmesh.variables['mbathy'][0,:,:] 
#used to calculate number of vertical ocean grid cells at each (i,j) (1=land point)
oldgdepw = oldmesh.variables['gdepw_0'][0,:,:,:]
oldsurface_tmask = oldmesh.variables['tmask'][0,0,:,:]
oldsurface_tmask = np.abs(oldsurface_tmask-1)
oldtmask = oldmesh.variables['tmask'][0,:,:,:]
oldtmask = np.abs(oldtmask-1)
oldmesh.close()

# Restart Files to Get Tracers (TS and Biology)

In [6]:
dataphys = nc.Dataset('/results/SalishSea/nowcast-green/06mar17/SalishSea_02002320_restart.nc')
databio = nc.Dataset('/results/SalishSea/nowcast-green/06mar17/SalishSea_02002320_restart_trc.nc')

In [7]:
physical = ['tn', 'sn']
biological = ['TRNDON', 'TRNMICZ','TRNMYRI','TRNNH4','TRNNO3','TRNTRA',
              'TRNPHY','TRNDIAT','TRNPON','TRNSi','TRNbSi']
varas = {}
for vb in physical:
    varas[vb] = dataphys.variables[vb][0, :]
for vb in biological:
    print (vb)
    varas[vb] = databio.variables[vb][0, :]
dataphys.close()
databio.close()
varall = physical + biological


TRNDON
TRNMICZ
TRNMYRI
TRNNH4
TRNNO3
TRNTRA
TRNPHY
TRNDIAT
TRNPON
TRNSi
TRNbSi


# Fill in any Missing Data Points

In [8]:
def find_mean(varas, varall, i, j, k, dd, oldtmask):
    for vb in varall:
        imin = max(i-dd, 0)
        imax = min(i+dd, 897)
        jmin = max(j-dd, 0)
        jmax = min(j+dd, 397)
        temporary = np.sum(varas[vb][k, imin:imax+1, jmin:jmax+1]*(1-oldtmask[k, imin:imax+1, jmin:jmax+1]))
        count = np.sum(1-oldtmask[k, imin:imax+1, jmin:jmax+1])
        if count == 0:
            varas[vb][k, i, j] = 0
        else:
            varas[vb][k, i, j] = temporary/count
    return varas

In [9]:
def fillit(kmax, oldtmask, varas, varall):
    dd = 1
    bad = 1
    while bad > 0:
        dd += 1
        good = 1
        while good > 0:
            good = 0; bad = 0; already = 0
            for k in range(kmax+1):
                for i in range(1, 898):
                    for j in range(1, 398):
                        if tmask[k,i,j] < oldtmask[k,i,j]:
                            if varas['sn'][k, i, j] > 0:
                                already = already + 1
                            else:
                                varas = find_mean(varas, varall, i, j, k, dd, oldtmask)
                                if varas['sn'][k, i, j] > 0:
                                    good = good + 1
                                else:
                                    bad = bad + 1
            print ('dd', dd, 'good', good)
            print ('already', already, 'bad', bad)
            



This can take a very long time if the bathymetries are very different, aka add a new long river.  If you want you can do it in pieces by starting with the first argument at say 5 and then slowly increasing it.  You do need to go to 39 finally. For new bathy201702 versus old bathy201702 its fast.

In [10]:
fillit(39, oldtmask, varas, varall)

dd 2 good 334
already 0 bad 0
dd 2 good 0
already 334 bad 0


# Write your Initial File

In [11]:
# build nc file
new_initialfile = nc.Dataset('Bathy201702_06mar17.nc', 'w')
nc_tools.init_dataset_attrs(
    new_initialfile, 
    title='All tracers for Bathymetry 201702 from nowcast-green 06mar17', 
    notebook_name='Tools/I_ForcingFiles/Initial/Initial_from_Restart_Bathy201702', 
    nc_filepath='sea_initial/v201702b/Bathy201702_06mar17.nc',
#    nc_filepath='tracers/initial/Bathy201702_18aug17.nc',
    comment='All Tracers, physical and biological')
new_initialfile.createDimension('y', 898)
new_initialfile.createDimension('x', 398)
new_initialfile.createDimension('deptht', 40)
new_initialfile.createDimension('time_counter', None)

file format: NETCDF4
Conventions: CF-1.6
title: All tracers for Bathymetry 201702 from nowcast-green 06mar17
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/salishsea/tools/src/tip/Tools/I_ForcingFiles/Initial/Initial_from_Restart_Bathy201702.ipynb
references: REQUIRED
history: [2018-04-14 18:08:40] Created netCDF4 zlib=True dataset.
comment: All Tracers, physical and biological


<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 0

In [12]:
thevara = {}
for vb in varall:
    thevara[vb] = new_initialfile.createVariable(
        vb, 'float32', ('time_counter', 'deptht', 'y', 'x'), zlib=True,
        least_significant_digit=1e-5, fill_value=-99)
    thevara[vb][0] = varas[vb]
    print (np.max(thevara[vb]))
new_initialfile

11.0
33.9375
17.5
0.6875
0.875
7.1875
47.25
5.8125
5.8125
2.125
1.375
68.6875
1.125


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    title: All tracers for Bathymetry 201702 from nowcast-green 06mar17
    institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
    source: https://bitbucket.org/salishsea/tools/src/tip/Tools/I_ForcingFiles/Initial/Initial_from_Restart_Bathy201702.ipynb
    references: REQUIRED
    history: [2018-04-14 18:08:40] Created netCDF4 zlib=True dataset.
    comment: All Tracers, physical and biological
    dimensions(sizes): y(898), x(398), deptht(40), time_counter(1)
    variables(dimensions): float32 tn(time_counter,deptht,y,x), float32 sn(time_counter,deptht,y,x), float32 TRNDON(time_counter,deptht,y,x), float32 TRNMICZ(time_counter,deptht,y,x), float32 TRNMYRI(time_counter,deptht,y,x), float32 TRNNH4(time_counter,deptht,y,x), float32 TRNNO3(time_counter,deptht,y,x), float32 TRNTRA(time_counter,deptht,y,x), float32 TRNPHY(time_counter,deptht,y,x

In [13]:
new_initialfile.close()